In [ ]:
!pip install openai==0.28
!pip install langchain
!pip install faiss-cpu
!pip install tiktoken
!pip install langchain_community
!pip install pypdf
!pip install transformers
!pip install sentence_transformers
!pip install slack_sdk
!pip install python-dotenv

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 5

In [ ]:
from dotenv import load_dotenv
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.schema import format_document
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import ChatPromptTemplate


from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings


load_dotenv()
gpt_model_name = 'gpt-4o-mini'



def retriever(file_path, k=2):
    assert file_path.endswith('.pdf'), "PDF files are the only supported format"
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()

    model_name = 'sentence-transformers/all-MiniLM-L6-v2'
    model_kwargs = {'device': 'cpu'}
    encode_kwargs = {'normalize_embeddings': True}
    embedding_model = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    faiss_index = FAISS.from_documents(pages, embedding_model)
    retriever = faiss_index.as_retriever(search_kwargs={"k": k})

    return retriever


standalone_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
                      Chat History:
                      {chat_history}
                      Follow Up Input: {question}
                      Standalone question:
                      """
STANDALONE_QUESTION_PROMPT = PromptTemplate.from_template(standalone_template)

answer_template = """Answer the question based only on the following context:
                    {context}

                    Question: {question}
                  """
ANSWER_PROMPT = ChatPromptTemplate.from_template(answer_template)
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")



def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    combined_docs=document_separator.join(doc_strings)
    return combined_docs




def llm_chain(retriever,memory):
    loaded_memory = RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
    )
    # Now we calculate the standalone question
    standalone_question = {
        "standalone_question": {
            "question": lambda x: x["question"],
            "chat_history": lambda x: x["chat_history"],
        }
        | STANDALONE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0,model=gpt_model_name)
        | StrOutputParser(),
    }
    # Now we retrieve the documents
    retrieved_documents = {
        "docs": itemgetter("standalone_question") | retriever,
        "question": lambda x: x["standalone_question"],
    }
    # Now we construct the inputs for the final prompt
    final_inputs = {
        "context": lambda x: _combine_documents(x["docs"]),
        "question": itemgetter("question"),
    }
    # And finally, we do the part that returns the answers
    answer = {
        "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(temperature=0,model=gpt_model_name),
        "docs": itemgetter("docs"),
    }
    # And now we put it all together!
    final_chain = loaded_memory | standalone_question | retrieved_documents | answer
    return final_chain

In [ ]:
file_path = '/content/handbook.pdf'
retriever_=retriever(file_path,k=2)
memory = ConversationBufferMemory()
chain=llm_chain(retriever_,memory)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

In [ ]:
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
import os


client = WebClient(token=os.getenv("SLACK_USER_TOKEN"))
def send_message_to_slack(channel_name, message):
    """
    Name:
      send_message_to_slack.
    Description:
      Used to send a message to a slack channel named 'channel_name'.
    Args:
      channel_name : Chanel name to send the message to
      message: The message to send
    Required:
      channel_name
      message

    Returns:
      A JSON string containing the channel_name and  message.

    """

    try:
        response = client.chat_postMessage(channel=channel_name, text=message)
        print("\tPosted Successfully!")
    except SlackApiError as e:
        print(f"Error posting message: {e.response['error']}")


def search_information_in_document(query):
    """
    Name:
      search_information_in_document.
    Description:
      Search any information in the document.
    Args:
      query : query to search
    Required:
      query

    Returns:
      A JSON string containing the query.

    """
    inputs = {"question": question}
    result = chain.invoke(inputs)
    memory.save_context(inputs, {"answer": result["answer"].content})

    return result["answer"].content


print(search_information_in_document.__doc__)
print(send_message_to_slack.__doc__)



    Name:
      search_information_in_document.
    Description:
      Search any information in the document.
    Args:
      query : query to search
    Required:
      query

    Returns:
      A JSON string containing the query.

    

    Name:
      send_message_to_slack.
    Description:
      Used to send a message to a slack channel named 'channel_name'.
    Args:
      channel_name : Chanel name to send the message to
      message: The message to send
    Required:
      channel_name
      message

    Returns:
      A JSON string containing the channel_name and  message.

    


In [ ]:
system_prompt = """
You are designed to help with a variety of tasks, To answer any query use the provide tools only.

## Tools
You have access to a wide variety of tools. You are responsible for using
the tools in any sequence you deem appropriate to complete the task at hand.

You have access to the following tools:
{tool_desc}

## Answer format
Answer the question by considering chat_history and observation. Answer should be in the following format.

1. Use this format if you need a tool to get some information before giving a final answer.
  Thought: I need to use a tool to help me answer the question.
  Action: Tool name without any quotes, if using a tool.
  Action Input: The input to the tool, in a JSON format representing the kwargs.

2. Use this format if either the final answer is there in the observation or you don't need any tool to get the final answer.
  Thought: I can answer without using any more tools.
  Answer: [Your answer here]

"""

user_prompt="""
### Chat History:
{chat_history}

### Observation:
{observation}

### Question :
{question}

"""


In [ ]:
import json
import re
def extract_tool_use(input_text):
    pattern = r"\s*Thought:(.*?)Action:(.*?)Action Input:(.*?)(?:\n|$)"

    match = re.search(pattern, input_text, re.DOTALL)
    if not match:
        raise ValueError(f"Could not extract tool use from input text: {input_text}")

    thought = match.group(1).strip()
    action = match.group(2).strip()
    action_input = match.group(3).strip()
    action_input=json.loads(action_input)
    return thought, action, action_input



In [ ]:
import openai
questions=["What is the name of the company, send this information to 'coding' slack channel",
           'Who is the CEO of the company?']

observation=''
chat_histroy=[]
tools={fun.__name__:fun for fun in [search_information_in_document, send_message_to_slack]}

system_message= system_prompt.format(tool_desc=[tools[tool].__doc__ for tool in tools])

for question in questions:
  print(f"\nQuestion: {question}")

  for _ in range(3):
    user_message=user_prompt.format(question=question,
                                    chat_history='\n'.join(chat_histroy),
                                    observation=observation)

    #print(f"User Message: \n {'*'*50} \n {user_message}")
    messages = [
      {"role": "system", "content":system_message},
      {"role": "user", "content": user_message}
      ]
    response = openai.ChatCompletion.create(
          model=gpt_model_name,
          messages=messages,
      )
    response=response["choices"][0]["message"]

    if "Action:" in response['content']:
      thought, action, action_input=extract_tool_use(response['content'])
      tool=tools[action]
      print(f"\tThought: {thought}")
      print(f"\tAction: {action}")
      print(f"\tAction Input: {action_input}")
      observation=tool(**action_input)

    else:
      thought, answer=response['content'].split('\n')
      thought=thought.split(':')[-1].strip()
      answer=answer.split(':')[-1].strip()
      print(f"\tThought: {thought}")
      print(f"\tAnswer: {answer}")
      break
    print('\n')

    chat_histroy.append(f"User: \n {question}")
    chat_histroy.append(f"Assistant: \n {response['content']}")




Question: What is the name of the company, send this information to 'coding' slack channel
	Thought: I need to use a tool to help me answer the question.
	Action: search_information_in_document
	Action Input: {'query': 'name of the company'}


	Thought: I can answer without using any more tools.
	Action: send_message_to_slack
	Action Input: {'channel_name': 'coding', 'message': 'The name of the company is Zania, Inc.'}
	Posted Successfully!


	Thought: I need to use a tool to help me answer the question.
	Action: search_information_in_document
	Action Input: {'query': 'name of the company'}



Question: Who is the CEO of the company?
	Thought: I need to use a tool to help me answer the question.
	Action: search_information_in_document
	Action Input: {'query': 'CEO of the company'}


	Thought: I can answer without using any more tools.
	Answer: The CEO of Zania, Inc. is Shruti Gupta.
